In [14]:
import pandas as pd
import pickle
import os
import numpy as np
import re

# 타격 세부 항목 파싱 함수
def parse_hit_details(etctext, hit_type):
    return etctext.count(hit_type) if isinstance(etctext, str) else 0


def find_info(df, hit_type):
    # 회차 열만 선택 (숫자로 된 열)
    inning_columns = [col for col in df.columns if str(col).strip().isdigit()]
    
    # 타격 종류에 따라 판별 방식 결정
    if hit_type == '2루타':
        suffix = '2'
        mode = 'endswith'
    elif hit_type == '3루타':
        suffix = '3'
        mode = 'endswith'
    elif hit_type == '병살타':
        suffix = '병'
        mode = 'endswith'
    elif hit_type == '볼넷':
        keyword = '4구'
        mode = 'contains'
    elif hit_type == '사구':
        keyword = '사구'
        mode = 'contains'
    elif hit_type == '희생플라이':
        keyword = '희비'
        mode = 'contains'
    else:
        raise ValueError(f"지원하지 않는 타격 종류입니다: {hit_type}")

    # 조건에 따라 카운트
    count = 0
    for col in inning_columns:
        col_data = df[col].astype(str)
        if mode == 'endswith':
            count += col_data.str.endswith(suffix).sum()
        elif mode == 'contains':
            count += col_data.str.contains(keyword).sum()
    
    return count

def get_total_home_runs(df):
    # '홈런' 열을 숫자로 변환
    df['홈런'] = pd.to_numeric(df['홈런'], errors='coerce')
    
    # NaN을 제외한 합계 계산
    return int(df['홈런'].sum(skipna=True))



def find_info2(df_etc, df_pit):
    # '폭투' 행 가져오기
    row = df_etc[df_etc['구분'] == '폭투']
    if row.empty or pd.isna(row.iloc[0]['내용']):
        return 0

    content = row.iloc[0]['내용']  # 폭투 내용 문자열

    # 패턴 1: 이름+숫자+괄호, 예: 윤규진3(8회)
    pattern_with_number = re.findall(r'([가-힣]+)(\d+)\([^)]*\)', content)
    
    # 패턴 2: 숫자 없이 이름만 있는 경우, 예: 문정(4회)
    pattern_without_number = re.findall(r'(?<!\d)([가-힣]+)\([^)]*\)', content)

    total = 0
    pit_names = set(df_pit['선수명'].values)

    # 이름 + 숫자 있는 경우
    for name, count in pattern_with_number:
        if name in pit_names:
            total += int(count)

    # 이름만 있고 숫자 없는 경우 → 1회로 간주
    for name in pattern_without_number:
        # 이름이 숫자 있는 케이스에서 이미 처리됐으면 제외
        if name in pit_names and name not in [n for n, _ in pattern_with_number]:
            total += 1

    return total



# 데이터 경로 설정
path = r'./DATA_15'
files = [f for f in os.listdir(path) if f.endswith('.pickle')]

# 결과 저장 리스트
all_data = []

# 파일 반복
for fname in files:
    with open(os.path.join(path, fname), 'rb') as f:
        game_dict = pickle.load(f)
        
        game_info = game_dict['GameInfo']
        game_date = game_info['GAME_DATE']
        year = int(game_date[:4])
        stadium = game_info['STADIUM']
        crowd = game_info.get('CROWD', '').replace(',', '')
        try:
            crowd = int(crowd)
        except:
            crowd = np.nan
        score_df = game_dict['Score']

        # ETC 텍스트
        etc_df = game_dict.get('ETC', pd.DataFrame())
        etc_text = ""
        if not etc_df.empty:
            etc_text = " ".join(etc_df['내용'].astype(str).tolist())

        # 팀별 처리
        for team_key, team_id, team_name, location in [
            ('Team1', game_info['AWAY_ID'], game_info['AWAY_NAME'], '원정'),
            ('Team2', game_info['HOME_ID'], game_info['HOME_NAME'], '홈')
        ]:
            hitters = game_dict.get(f'{team_key}_Hitter', pd.DataFrame())
            pitcher_df = game_dict.get(f'{team_key}_Pitcher', pd.DataFrame())

            if team_key == 'Team1':
                other_team_key = 'Team2'
            else:
                other_team_key = 'Team1'

            pitcher_df_other = game_dict.get(f'{other_team_key}_Pitcher', pd.DataFrame())

            # 점수 데이터 필터링
            try:
                score_row = score_df[score_df['팀명'] == team_id].iloc[0]
                runs = int(score_row['R'])
            except:
                runs = np.nan

            # 타자 통계 계산
            try:
                hitters['타수'] = pd.to_numeric(hitters['타수'], errors='coerce')
                hitters['안타'] = pd.to_numeric(hitters['안타'], errors='coerce')
                hitters['타점'] = pd.to_numeric(hitters['타점'], errors='coerce')
                hitters['타율'] = pd.to_numeric(hitters['타율'], errors='coerce')
                hitters['출루율'] = pd.to_numeric(hitters.get('출루율', np.nan), errors='coerce')
                hitters['장타율'] = pd.to_numeric(hitters.get('장타율', np.nan), errors='coerce')

                ab = hitters['타수'].sum()
                hits = hitters['안타'].sum()
                rbi = hitters['타점'].sum()
                avg = hits / ab if ab > 0 else np.nan
                obp = hitters['출루율'].mean() if not hitters['출루율'].isna().all() else np.nan
                slg = hitters['장타율'].mean() if not hitters['장타율'].isna().all() else np.nan
            except:
                ab = hits = rbi = avg = obp = slg = np.nan

            # 투수 통계
            try:
                pitcher_df['4사구'] = pd.to_numeric(pitcher_df.get('4사구', np.nan), errors='coerce')
                pitcher_df['삼진'] = pd.to_numeric(pitcher_df.get('삼진', np.nan), errors='coerce')
                bb_hbp = pitcher_df['4사구'].sum()
                so = pitcher_df['삼진'].sum()
            except:
                bb_hbp = so = np.nan

            # ETC 기반 이벤트 파싱
            two_b = find_info(hitters, '2루타')
            three_b = find_info(hitters, '3루타')
            double_play = find_info(hitters, '병살타')
            wild_pitch = find_info2(etc_df, pitcher_df)
            base_on_balls = find_info(hitters, '볼넷')
            hit_by_pitch = find_info(hitters, '사구')
            sacrifice_flies = find_info(hitters, '희생플라이')
            home_runs = get_total_home_runs(pitcher_df_other)

            # 1루타 계산
            one_b = hits - two_b - three_b - home_runs

            # 출루율 계산
            obp_denominator = ab + base_on_balls + hit_by_pitch + sacrifice_flies
            obp_numerator = hits + base_on_balls + hit_by_pitch
            obp = obp_numerator / obp_denominator if obp_denominator > 0 else np.nan

            # 장타율 계산
            total_bases = one_b + 2 * two_b + 3 * three_b + 4 * home_runs
            slg = total_bases / ab if ab > 0 else np.nan

            # 단타율 계산
            singles_rate = one_b / ab if ab > 0 else np.nan


            # 결과 저장
            all_data.append({
                '날짜': game_date,
                '연도': year,
                '경기장': stadium,
                '팀명': team_name,
                '홈/원정': location,
                '타수': ab,
                '안타': hits,
                '득점': runs,
                '2루타': two_b,
                '3루타': three_b,
                '홈런': home_runs,
                '볼넷': base_on_balls,
                '사구': hit_by_pitch,
                '희생플라이': sacrifice_flies,
                '타점': rbi,
                '병살타': double_play,
                '폭투': wild_pitch,
                '관중수': crowd,
                '타율': round(avg, 3) if not np.isnan(avg) else np.nan,
                '출루율': round(obp, 3) if not np.isnan(obp) else np.nan,
                '장타율': round(slg, 3) if not np.isnan(slg) else np.nan,
                '단타율': round(singles_rate, 3) if not np.isnan(singles_rate) else np.nan
            })

# 데이터프레임 생성
df = pd.DataFrame(all_data)

# 날짜 컬럼을 datetime 형식으로 변환
df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce')

# 날짜 기준 오름차순 정렬
df = df.sort_values(by='날짜').reset_index(drop=True)

output_file = r'./BASEBALL_stats_15.xlsx'
df.to_excel(output_file, index=False)

In [15]:
df

,날짜,연도,경기장,팀명,홈/원정,타수,안타,득점,2루타,3루타,...,사구,희생플라이,타점,병살타,폭투,관중수,타율,출루율,장타율,단타율
0,2015-03-28,2015,사직,롯데,홈,37,14,12,5,0,...,1,0,12,1,0,27500,0.378,0.439,0.676,0.189
1,2015-03-28,2015,잠실,NC,원정,31,9,4,0,0,...,1,2,4,1,1,21746,0.290,0.368,0.290,0.290
2,2015-03-28,2015,잠실,두산,홈,35,12,9,0,1,...,0,2,9,0,0,21746,0.343,0.405,0.571,0.257
3,2015-03-28,2015,목동,한화,원정,42,10,4,2,0,...,1,1,4,0,1,12500,0.238,0.353,0.286,0.190
4,2015-03-28,2015,목동,넥센,홈,40,8,5,2,0,...,0,0,5,1,0,12500,0.200,0.304,0.400,0.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14677,2025-04-27,2025,대전(신),KT,원정,31,6,3,2,0,...,1,0,3,2,0,17000,0.194,0.324,0.258,0.129
14678,2025-04-27,2025,잠실,롯데,원정,32,7,4,3,0,...,1,0,4,2,1,23750,0.219,0.306,0.406,0.094
14679,2025-04-27,2025,잠실,두산,홈,34,13,13,4,0,...,1,1,13,2,0,23750,0.382,0.522,0.588,0.235
14680,2025-04-27,2025,광주,LG,원정,30,5,2,1,0,...,0,0,1,2,0,20500,0.167,0.286,0.200,0.133
